In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
import numpy as np
from gridworld import standard_grid, ACTION_SPACE
from iterative_policy_evaluation_deterministic import print_values, print_policy

In [53]:
SMALL_ENOUGH = 1e-3
GAMMA = 0.9

In [54]:
# copied from iterative_policy_evaluation
def get_transition_probs_and_rewards(grid):
  ### define transition probabilities and grid ###
  # the key is (s, a, s'), the value is the probability
  # that is, transition_probs[(s, a, s')] = p(s' | s, a)
  # any key NOT present will considered to be impossible (i.e. probability 0)
  transition_probs = {}

  # to reduce the dimensionality of the dictionary, we'll use deterministic
  # rewards, r(s, a, s')
  # note: you could make it simpler by using r(s') since the reward doesn't
  # actually depend on (s, a)
  rewards = {}

  for i in range(grid.rows):
    for j in range(grid.cols):
      s = (i, j)
      if not grid.is_terminal(s):
        for a in ACTION_SPACE:
          s2 = grid.get_next_state(s, a)
          transition_probs[(s, a, s2)] = 1
          if s2 in grid.rewards:
            rewards[(s, a, s2)] = grid.rewards[s2]

  return transition_probs, rewards

In [55]:
def evaluate_deterministic_policy(grid, policy, initV=None):
  # initialize V(s) = 0
  if initV is None:
    V = {}
    for s in grid.all_states():
      V[s] = 0
  else:
    # it's faster to use the existing V(s) since the value won't change
    # that much from one policy to the next
    V = initV

  # repeat until convergence
  it = 0
  while True:
    biggest_change = 0
    for s in grid.all_states():
      if not grid.is_terminal(s):
        old_v = V[s]
        new_v = 0 # we will accumulate the answer
        for a in ACTION_SPACE:
          for s2 in grid.all_states():

            # action probability is deterministic
            action_prob = 1 if policy.get(s) == a else 0
            
            # reward is a function of (s, a, s'), 0 if not specified
            r = rewards.get((s, a, s2), 0)
            new_v += action_prob * transition_probs.get((s, a, s2), 0) * (r + GAMMA * V[s2])

        # after done getting the new value, update the value table
        V[s] = new_v
        biggest_change = max(biggest_change, np.abs(old_v - V[s]))
    it += 1

    if biggest_change < SMALL_ENOUGH:
      break
  return V

In [56]:
grid = standard_grid()
transition_probs, rewards = get_transition_probs_and_rewards(grid)
# print rewards
print("rewards:")
print_values(grid.rewards, grid)

rewards:
---------------------------
 0.00| 0.00| 0.00| 1.00|
---------------------------
 0.00| 0.00| 0.00|-1.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|


In [57]:
# state -> action
# we'll randomly choose an action and update as we learn
policy = {}
for s in grid.actions.keys():
    policy[s] = np.random.choice(ACTION_SPACE)

# initial policy
print("initial policy:")
print_policy(policy, grid)


initial policy:
---------------------------
  U  |  R  |  U  |     |
---------------------------
  L  |     |  L  |     |
---------------------------
  U  |  R  |  R  |  U  |


In [58]:
# repeat until convergence - will break out when policy does not change
V = None
while True:

    # policy evaluation step - we already know how to do this!
    V = evaluate_deterministic_policy(grid, policy, initV=V)

    # policy improvement step
    is_policy_converged = True
    for s in grid.actions.keys():
        old_a = policy[s]
        new_a = None
        best_value = float('-inf')

        # loop through all possible actions to find the best current action
        for a in ACTION_SPACE:
            v = 0
            for s2 in grid.all_states():
                # reward is a function of (s, a, s'), 0 if not specified
                r = rewards.get((s, a, s2), 0)
                v += transition_probs.get((s, a, s2), 0) * (r + GAMMA * V[s2])

            if v > best_value:
                best_value = v
                new_a = a

        # new_a now represents the best action in this state
        policy[s] = new_a
        if new_a != old_a:
            is_policy_converged = False

    if is_policy_converged:
        break

    # once we're done, print the final policy and values
    print("values:")
    print_values(V, grid)
    print("policy:")
    print_policy(policy, grid)

values:
---------------------------
 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00|-0.81|-0.90|-1.00|
policy:
---------------------------
  U  |  U  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  L  |  U  |  L  |
values:
---------------------------
 0.00| 0.00| 1.00| 0.00|
---------------------------
 0.00| 0.00| 0.90| 0.00|
---------------------------
 0.00| 0.00| 0.81| 0.73|
policy:
---------------------------
  U  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  R  |  U  |  L  |
values:
---------------------------
 0.00| 0.90| 1.00| 0.00|
---------------------------
 0.00| 0.00| 0.90| 0.00|
---------------------------
 0.00| 0.73| 0.81| 0.73|
policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  R  |  R  |  U  |  L  

# Policy Iteration (Probabilistic)

In [65]:
from gridworld import windy_grid, windy_grid_penalized, ACTION_SPACE
from iterative_policy_evaluation_probabilistic import print_values, print_policy

In [66]:
# copied from iterative_policy_evaluation
def get_transition_probs_and_rewards(grid):
  ### define transition probabilities and grid ###
  # the key is (s, a, s'), the value is the probability
  # that is, transition_probs[(s, a, s')] = p(s' | s, a)
  # any key NOT present will considered to be impossible (i.e. probability 0)
  transition_probs = {}

  # to reduce the dimensionality of the dictionary, we'll use deterministic
  # rewards, r(s, a, s')
  # note: you could make it simpler by using r(s') since the reward doesn't
  # actually depend on (s, a)
  rewards = {}

  for (s, a), v in grid.probs.items():
    for s2, p in v.items():
      transition_probs[(s, a, s2)] = p
      rewards[(s, a, s2)] = grid.rewards.get(s2, 0)

  return transition_probs, rewards

In [67]:
grid = windy_grid_penalized(-0.1)
# grid = windy_grid()
transition_probs, rewards = get_transition_probs_and_rewards(grid)
# print rewards
print("rewards:")
print_values(grid.rewards, grid)

rewards:
---------------------------
-0.10|-0.10|-0.10| 1.00|
---------------------------
-0.10| 0.00|-0.10|-1.00|
---------------------------
-0.10|-0.10|-0.10|-0.10|


In [68]:
 # state -> action
# we'll randomly choose an action and update as we learn
policy = {}
for s in grid.actions.keys():
    policy[s] = np.random.choice(ACTION_SPACE)

# initial policy
print("initial policy:")
print_policy(policy, grid)

initial policy:
---------------------------
  R  |  L  |  R  |     |
---------------------------
  U  |     |  D  |     |
---------------------------
  R  |  D  |  D  |  U  |


In [69]:
# repeat until convergence - will break out when policy does not change
V = None
while True:
    # policy evaluation step - we already know how to do this!
    V = evaluate_deterministic_policy(grid, policy, initV=V)

    # policy improvement step
    is_policy_converged = True
    for s in grid.actions.keys():
        old_a = policy[s]
        new_a = None
        best_value = float('-inf')

        # loop through all possible actions to find the best current action
        for a in ACTION_SPACE:
            v = 0
            for s2 in grid.all_states():
                # reward is a function of (s, a, s'), 0 if not specified
                r = rewards.get((s, a, s2), 0)
                v += transition_probs.get((s, a, s2), 0) * (r + GAMMA * V[s2])

            if v > best_value:
                best_value = v
                new_a = a

        # new_a now represents the best action in this state
        policy[s] = new_a
        if new_a != old_a:
            is_policy_converged = False
        
    print("policy:")
    print_policy(policy, grid)
    print("values:")
    print_values(V, grid)
    if is_policy_converged:
        print("======================================================")
        break

# once we're done, print the final policy and values
print("policy:")
print_policy(policy, grid)
print("values:")
print_values(V, grid)

policy:
---------------------------
  D  |  R  |  R  |     |
---------------------------
  D  |     |  U  |     |
---------------------------
  R  |  U  |  U  |  L  |
values:
---------------------------
-1.00|-1.00| 1.00| 0.00|
---------------------------
-1.00| 0.00|-0.99| 0.00|
---------------------------
-0.99|-0.99|-0.99|-1.00|
policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  D  |     |  U  |     |
---------------------------
  R  |  R  |  U  |  L  |
values:
---------------------------
-0.99| 0.80| 1.00| 0.00|
---------------------------
-0.99| 0.00|-0.10| 0.00|
---------------------------
-0.99|-0.99|-0.19|-0.27|
policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  R  |  R  |  U  |  L  |
values:
---------------------------
 0.62| 0.80| 1.00| 0.00|
---------------------------
-0.41| 0.00|-0.10| 0.00|
---------------------------
-0.34|-0.27|-0.19|-0.27